<a href="https://colab.research.google.com/github/Kashara-Alvin-Ssali/ML-models/blob/main/TRestNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
pip install tensorflow keras numpy matplotlib


In [26]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from google.colab import files
from sklearn.utils.class_weight import compute_class_weight

In [27]:
from google.colab import drive  # <-- Import drive
drive.mount('/content/drive') # Mount to the default path without spaces

# Now access your dataset using the full path:
train_dir = "/content/drive/MyDrive/Dataset4/Training"
validation_dir = "/content/drive/MyDrive/Dataset4/Validation"
test_dir = "/content/drive/MyDrive/Dataset4/Testing"  # If needed for testing

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# train_dir = "./Dataset4/Training"
# validation_dir = "./Dataset4/Validation"
# test_dir = "./Dataset4/Testing"  # If needed for testing


In [29]:
# Image dimensions
height = 300
width = 300
batch_size = 8


In [30]:
# Load ResNet50 without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(height, width, 3))

In [31]:
# Data

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,  # Add shear
    zoom_range=0.2,   # Add zoom
    brightness_range=[0.8, 1.2]  # Add brightness variation
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
 # Added test data generator

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = dict(enumerate(class_weights))
validation_generator = val_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 64 images belonging to 2 classes.
Found 64 images belonging to 2 classes.
Found 6 images belonging to 3 classes.


In [32]:
# Add custom layers to ResNet50
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

In [33]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
# # Callbacks
# checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [37]:
model.fit(
    train_generator,
    validation_data=train_generator,
    epochs=12,
    # callbacks=[checkpoint, early_stopping],
    class_weight=class_weights  # Add class weights
)

Epoch 1/12
8/8 ━━━━━━━━━━━━━━━━━━━━ 140s 18s/step - accuracy: 0.5065 - loss: 0.6969 - val_accuracy: 0.5000 - val_loss: 6.4018
Epoch 2/12
8/8 ━━━━━━━━━━━━━━━━━━━━ 117s 15s/step - accuracy: 0.6533 - loss: 0.7102 - val_accuracy: 0.4531 - val_loss: 1.3052
Epoch 3/12
8/8 ━━━━━━━━━━━━━━━━━━━━ 117s 15s/step - accuracy: 0.5311 - loss: 0.7293 - val_accuracy: 0.5000 - val_loss: 1.2567
Epoch 4/12
8/8 ━━━━━━━━━━━━━━━━━━━━ 116s 15s/step - accuracy: 0.5780 - loss: 0.6525 - val_accuracy: 0.4531 - val_loss: 4.0973
Epoch 5/12
8/8 ━━━━━━━━━━━━━━━━━━━━ 115s 15s/step - accuracy: 0.5089 - loss: 1.3665 - val_accuracy: 0.5938 - val_loss: 1.5129
Epoch 6/12
8/8 ━━━━━━━━━━━━━━━━━━━━ 161s 18s/step - accuracy: 0.5116 - loss: 0.8058 - val_accuracy: 0.6250 - val_loss: 1.0232
Epoch 7/12
8/8 ━━━━━━━━━━━━━━━━━━━━ 116s 15s/step - accuracy: 0.5094 - loss: 0.7583 - val_accuracy: 0.5312 - val_loss: 0.9714
Epoch 8/12
8/8 ━━━━━━━━━━━━━━━━━━━━ 133s 17s/step - accuracy: 0.4852 - loss: 0.7251 - val_accuracy: 0.5156 - val_loss:

In [38]:
# Save the final model
model.save("resnet50_model.h5")
print("Model training complete and saved as resnet50_model.h5")

Model training complete and saved as resnet50_model.h5


In [45]:
test_loss, test_acc = model.evaluate(train_generator)
print(f"Test Accuracy: {test_acc:.2%}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.5952 - loss: 0.6500
Test Accuracy: 59.38%


In [44]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = "/content/drive/MyDrive/Dataset4/Training/Fake/14.jpg"  # Corrected path
img = image.load_img(img_path, target_size=(300, 300))  # Match model input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = preprocess_input(img_array)  # Apply same preprocessing as training

# Make a prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)  # Get class index
class_labels = list(train_generator.class_indices.keys())  # Get class names
print(f"Predicted Class: {class_labels[predicted_class]}")  # Print the class label

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 362ms/step
Predicted Class: Real
